# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

----
Import establishments.json via terminal

`mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

----

In [22]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [23]:
# Create an instance of MongoClient
mongo = MongoClient(port = 27017)

In [24]:
#   List databases to ensure uk_food database is there
mongo.list_database_names()

['admin', 'config', 'local', 'uk_food']

In [25]:
# assign the uk_food database to a variable name
uk_food = mongo["uk_food"]

In [26]:
# review the collections in our database
uk_food.collection_names()

/Users/bilal/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  


['establishments']

In [27]:
#   Pretty print one document from establishments
pprint(uk_food.establishments.find_one())

{'AddressLine1': 'West Cottage',
 'AddressLine2': 'South Foreland Lighthouse',
 'AddressLine3': 'The Front',
 'AddressLine4': 'St Margarets Bay',
 'BusinessName': 'Mrs Knotts Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.492467848017,
 'FHRSID': 429991,
 'LocalAuthorityBusinessID': 'PI/000055200',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6HP',
 'RatingDate': '2019-05-02T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63f041be5bdff9ae39fbeb99'),
 'geocode': {'latitude': '51.14061', 'longitude': '1.371894'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/429991',
            'rel': 'self'}],
 'meta': {'dataSource': None,
  

In [28]:
# assign the collection to a variable
establishments = uk_food.establishments

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [29]:
# Create a dictionary for the new restaurant data
restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [31]:
# Insert the new restaurant into the collection
establishments.insert_one(restaurant)

In [32]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}


pprint(establishments.find_one(query))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63f0437ef93f7a08f98a9258'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [33]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID":1, "BusinessType":1}

pprint(establishments.find_one(query, projection = fields))

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63f041be5bdff9ae39fbeb99')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [34]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName":"Penang Flavours"},
                            {"$set":{"BusinessTypeID":1}})

In [35]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
fields = {"BusinessName":1, "BusinessType":1, "BusinessTypeID":1}

pprint(establishments.find_one(query, fields))

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63f0437ef93f7a08f98a9258')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [36]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName":"Dover"}

pprint(establishments.count_documents(query))

994


In [37]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

In [38]:
# Check if any remaining documents include Dover
pprint(establishments.count_documents(query))

0


In [39]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one({}))

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63f041be5bdff9ae39fbee81'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [40]:
# Change the data type from String to Decimal for longitude

establishments.update_many({},
    [{"$set":{"geocode.longitude":{"$toDouble":"$geocode.longitude"}}}])

In [41]:
# Change the data type from String to Decimal for latitude
establishments.update_many({},
    [{"$set":{"geocode.latitude":{"$toDouble":"$geocode.latitude"}}}])

In [42]:
# Check that the coordinates are now numbers
fields = {"geocode.latitude":1, "geocode.longitude":1}

pprint(establishments.find_one({},fields))

{'_id': ObjectId('63f041be5bdff9ae39fbee81'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408}}
